In [7]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import logging
logging.set_verbosity_error()

In [5]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
df['label'] = df['target'].apply(lambda x: 0 if x == 0 else 1) 
df

,target,id,date,flag,user,text,label
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",0
...,...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,1
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,1
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,1
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,1


In [8]:
df_sample = df.sample(n=10000, random_state=42)
texts = df_sample['text'].tolist()
labels = df_sample['label'].tolist()


In [13]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

import warnings
warnings.filterwarnings('ignore')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [15]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [19]:
batch_size = 8
epochs = 2

model.fit(
    train_dataset.shuffle(1000).batch(batch_size),
    epochs=epochs,
    batch_size=batch_size
)

Epoch 1/2

1000/1000 [==============================] - 18521s 19s/step - loss: 0.4907 - accuracy: 0.7650
Epoch 2/2
1000/1000 [==============================] - 1183s 1s/step - loss: 0.3203 - accuracy: 0.8675


In [20]:
results = model.evaluate(test_dataset.batch(batch_size))
print(f"\n Test Loss: {results[0]:.4f}")
print(f" Test Accuracy: {results[1]:.4f}")

250/250 [==============================] - 57s 222ms/step - loss: 0.4678 - accuracy: 0.7855

 Test Loss: 0.4678
 Test Accuracy: 0.7855


In [23]:
predictions = model.predict(test_dataset.batch(batch_size)).logits
predicted_labels = np.argmax(predictions, axis=1)

print("\n Sample Predictions:")
for i in range(5):
    print(f"\nTweet: {X_test[i]}")
    print(f"Actual Label: {y_test[i]}")
    print(f"Predicted Label: {predicted_labels[i]}")

250/250 [==============================] - 56s 220ms/step

 Sample Predictions:

Tweet: @gunsandbutter12 o yeah!!! I remeber u weren't happy bout that 
Actual Label: 0
Predicted Label: 0

Tweet: @AAA_Amerah - good morning, Amerah! 
Actual Label: 1
Predicted Label: 1

Tweet: Did @KylePetty just say he drives down the freeway with his knee while tweeting? 
Actual Label: 1
Predicted Label: 1

Tweet: @SplashTurnTwist Well least it's not hand measles! You got physics in the morning? 
Actual Label: 0
Predicted Label: 1

Tweet: good night &amp; gute nacht! 
Actual Label: 1
Predicted Label: 1
